### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [1]:
from datetime import datetime, timezone, timedelta

class Account:
    _MONTHLY_INTEREST = 1.05
    _LAST_TRANSACTION_ID = 0
    DATE_FORMAT = "%Y%m%d%H%M%S"
    
    def __init__(self, accountnumber: int, first_name: str, last_name: str, tz_offset: int = 0):
        self._account_number: int = accountnumber
        self._balance: float = 0.0
        self._first_name: str = first_name
        self._last_name: str = last_name
        self._tz_offset: int = tz_offset
            
            
    @property
    def balance(self):
        return self._balance
    
    
    @property
    def full_name(self):
        return f"{self._first_name} {self._last_name}"
    
    
    @property
    def tz_offset(self):
        return self._tz_offset
        
    
    def deposit(self, amount: float):
        self._balance += amount
        return Account.get_confirmation_number('D', self._account_number)
        
        
    def withdraw(self, amount: float):
        if self.balance - amount < 0:
            return Account.get_confirmation_number('X', self._account_number)
        
        self._balance -= amount
        return Account.get_confirmation_number('W', self._account_number)
        
        
    def deposit_interest(self):
        self._balance = round(self._balance * self._MONTHLY_INTEREST, 2)
        return Account.get_confirmation_number('I', self._account_number)
    
    
    @staticmethod
    def _validate_transaction_type(transaction_type: str):
        types = ['D', 'W', 'I', 'X']
        if transaction_type not in types:
            type_string = ""
            for value in types:
                if len(type_string) != 0:
                    type_string += ", "
                type_string += str(value)
            raise ValueError(f"The transaction type must be one of the following: {type_string}.")
        
    
    @classmethod
    def get_confirmation_number(cls, type: str, accountnumber: int):
        Account._validate_transaction_type(type)
            
        timestamp = datetime.now(timezone.utc).strftime(Account.DATE_FORMAT)
        
        cls._LAST_TRANSACTION_ID += 1
        transaction_id = cls._LAST_TRANSACTION_ID
        
        return f"{type}-{str(accountnumber)}-{timestamp}-{transaction_id}"
    
    
    def get_transaction_dict(self, transaction_code: str):
        parts = transaction_code.strip().split("-")
        if len(parts) != 4:
            raise ValueError("Invalid transaction code: the code must have a transaction type, account number, timestamp and transaction id separated by a hyphen.")
        
        result = {}
        
        transaction_type = str(parts[0])
        Account._validate_transaction_type(transaction_type)
        result["transaction_type"] = transaction_type
        
        try:
            account_number = int(parts[1])
        except ValueError as e:
            raise ValueError("Invalid transaction code: the account number is not an integer")
        result["account_number"] = account_number
        
        
        try:
            transaction_id = int(parts[3])
        except ValueError as e:
            raise ValueError("Invalid transaction code: the transaction ID is not an integer")
        result["transaction_id"] = transaction_id
        
        try:
            date_object = datetime.strptime(parts[2], Account.DATE_FORMAT).replace(tzinfo=timezone.utc)
        except ValueError as e:
            raise ValueError(f"Invalid transaction code: the timestamp is incorrectly formatted, should be {Account.DATE_FORMAT}.")
        result["time_utc"] = date_object.isoformat()
        tz = timezone(timedelta(hours=self.tz_offset))
        result["time"] = date_object.astimezone(tz).isoformat()
        
        return result

In [2]:
a = Account(128768, "Axel", "Kohler", +2)
a, a.full_name, a.balance

(<__main__.Account at 0x7f195052feb0>, 'Axel Kohler', 0.0)

In [3]:
b = Account(61278, "Janine", "Abbring", 5)
b

In [4]:
a.balance

0.0

In [5]:
t1 = a.deposit(3000)

In [6]:
t2 = a.withdraw(50)

In [7]:
a._balance

2950.0

In [8]:
a.withdraw(2950)

'W-128768-20230718085311-3'

In [9]:
a._balance

0.0

In [10]:
a.withdraw(1)

'X-128768-20230718085311-4'

In [11]:
a._balance

0.0

In [12]:
a.deposit(100)

'D-128768-20230718085311-5'

In [13]:
a._balance

100.0

In [14]:
t3 = a.withdraw(150)

In [15]:
a._balance

100.0

In [16]:
t4 = a.deposit_interest()

In [17]:
a._balance

105.0

In [18]:
t5 = b.deposit(1200)

In [19]:
a.get_transaction_dict(t1)

{'transaction_type': 'D',
 'account_number': 128768,
 'transaction_id': 1,
 'time_utc': '2023-07-18T08:53:11+00:00',
 'time': '2023-07-18T10:53:11+02:00'}

In [20]:
a.get_transaction_dict(t2)

{'transaction_type': 'W',
 'account_number': 128768,
 'transaction_id': 2,
 'time_utc': '2023-07-18T08:53:11+00:00',
 'time': '2023-07-18T10:53:11+02:00'}

In [21]:
a.get_transaction_dict(t3)

{'transaction_type': 'X',
 'account_number': 128768,
 'transaction_id': 6,
 'time_utc': '2023-07-18T08:53:12+00:00',
 'time': '2023-07-18T10:53:12+02:00'}

In [22]:
a.get_transaction_dict(t4)

{'transaction_type': 'I',
 'account_number': 128768,
 'transaction_id': 7,
 'time_utc': '2023-07-18T08:53:12+00:00',
 'time': '2023-07-18T10:53:12+02:00'}

In [23]:
b.get_transaction_dict(t5)

{'transaction_type': 'D',
 'account_number': 61278,
 'transaction_id': 8,
 'time_utc': '2023-07-18T08:53:12+00:00',
 'time': '2023-07-18T13:53:12+05:00'}